In [5]:
# ✅ Brain Tumor Detection GUI with Grad-CAM and Detailed PDF Report

import tkinter as tk
from tkinter import filedialog, messagebox, scrolledtext, ttk
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
import cv2
import os
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
from datetime import datetime

# ==================== CONFIG ======================
MODEL_PATH = "brain_tumor_model.h5"  # <-- Update with your model path
CLASS_NAMES = ["glioma", "meningioma", "notumor", "pituitary"]
IMAGE_SIZE = (224, 224)

# ================== LOAD MODEL =====================
model = tf.keras.models.load_model(r"C:\Users\rushi\OneDrive\Desktop\Brain Tumor Detection\brain_tumor_cnn_best_model.h5")

# ================= EXPLANATION =====================
def explain_prediction(pred_class, confidence_all):
    explanation = ""
    if pred_class == "glioma":
        explanation += "Glioma Tumor Detected\n- Gliomas start in the glial cells of the brain.\n- The model focused on deeper brain regions.\n- Typical glioma signs seen in this scan.\n"
        explanation += "\nWhy NOT Meningioma?\n- Meningiomas are usually near skull edges, which was not seen."
        explanation += "\nWhy NOT Pituitary?\n- Pituitary tumors are in the lower center; this was not prominent here."
    elif pred_class == "meningioma":
        explanation += "Meningioma Tumor Detected\n- Meningiomas are extra-axial tumors attached to dura.\n- Model highlighted peripheral brain regions.\n"
        explanation += "\nWhy NOT Glioma?\n- No deep white matter activation observed."
    elif pred_class == "pituitary":
        explanation += "Pituitary Tumor Detected\n- Tumor is located in the sellar region.\n- Model focused on lower midline area.\n"
        explanation += "\nWhy NOT Glioma?\n- Gliomas affect lobes; here, the focus is central."
    else:
        explanation += "No Tumor Detected\n- No abnormal activation patterns.\n"
        explanation += "\nWhy NOT Others?\n- No signs matching glioma, meningioma, or pituitary tumors."

    explanation += "\n\nConfidence Scores:\n"
    for i, cls in enumerate(CLASS_NAMES):
        explanation += f"- {cls.title()}: {confidence_all[i]:.2f}%\n"
    return explanation

# =============== IMAGE PROCESSING ==================
def preprocess_image(img_path):
    img = Image.open(img_path).resize(IMAGE_SIZE)
    img_arr = np.array(img)
    if img_arr.ndim == 2:
        img_arr = np.stack([img_arr]*3, axis=-1)
    elif img_arr.shape[2] == 1:
        img_arr = np.repeat(img_arr, 3, axis=-1)
    img_arr = img_arr / 255.0
    img_arr = np.expand_dims(img_arr, axis=0)
    return img, img_arr

# ================ GRAD-CAM =========================
def generate_grad_cam(model, img_array, class_index, layer_name="conv2d"):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        loss = predictions[:, class_index]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = np.mean(conv_outputs * pooled_grads, axis=-1)
    heatmap = np.maximum(heatmap, 0)
    heatmap /= (tf.reduce_max(heatmap) + 1e-8)
    heatmap = cv2.resize(heatmap.numpy(), IMAGE_SIZE)
    heatmap_colored = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    return heatmap_colored

# ================ REPORT GENERATION ================
def save_pdf_report(img_path, pred_class, confidence, confidence_all, explanation, heatmap):
    save_path = filedialog.asksaveasfilename(defaultextension=".pdf", filetypes=[("PDF files", "*.pdf")])
    if not save_path:
        return

    c = canvas.Canvas(save_path, pagesize=letter)
    c.setFont("Helvetica-Bold", 14)
    c.drawString(30, 750, "Brain Tumor Detection Report")
    c.setFont("Helvetica", 10)
    c.drawString(30, 730, f"Prediction: {pred_class.title()} ({confidence:.2f}%)")
    c.drawString(30, 715, f"Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

    img = Image.open(img_path).resize((150, 150))
    img.save("temp_img.png")
    c.drawImage("temp_img.png", 30, 540, width=150, height=150)

    cv2.imwrite("temp_heatmap.jpg", heatmap)
    c.drawImage("temp_heatmap.jpg", 200, 540, width=150, height=150)

    c.setFont("Helvetica-Bold", 12)
    c.drawString(30, 500, "Explanation:")
    c.setFont("Helvetica", 9)
    text = c.beginText(30, 480)
    for line in explanation.split("\n"):
        text.textLine(line)
    c.drawText(text)

    c.save()
    os.remove("temp_img.png")
    os.remove("temp_heatmap.jpg")
    messagebox.showinfo("Saved", f"Report saved as: {save_path}")

# ===================== GUI =========================
def run_gui():
    root = tk.Tk()
    root.title("Brain Tumor Detector")
    root.geometry("700x600")
    root.configure(bg="#f2f2f2")

    result_var = tk.StringVar()
    confidence_var = tk.StringVar()

    def upload_image():
        file_path = filedialog.askopenfilename(filetypes=[("Images", "*.jpg *.png *.jpeg")])
        if not file_path:
            return

        img_display, img_array = preprocess_image(file_path)
        preds = model.predict(img_array)[0]
        class_index = np.argmax(preds)
        pred_class = CLASS_NAMES[class_index]
        confidence = preds[class_index] * 100

        heatmap = generate_grad_cam(model, img_array, class_index)

        img_display = img_display.resize((200, 200))
        img_tk = ImageTk.PhotoImage(img_display)
        image_label.config(image=img_tk)
        image_label.image = img_tk

        result_var.set(f"Prediction: {pred_class.title()} ({confidence:.2f}%)")
        explanation = explain_prediction(pred_class, preds * 100)
        explanation_box.delete(1.0, tk.END)
        explanation_box.insert(tk.END, explanation)

        download_button.config(state=tk.NORMAL)
        download_button.config(command=lambda: save_pdf_report(file_path, pred_class, confidence, preds * 100, explanation, heatmap))

    # ==== Widgets ====
    tk.Label(root, text="Brain Tumor Detection", font=("Helvetica", 16, "bold"), bg="#f2f2f2").pack(pady=10)
    tk.Button(root, text="Upload MRI Image", command=upload_image, bg="#4CAF50", fg="white", font=("Arial", 12)).pack()
    tk.Label(root, textvariable=result_var, font=("Arial", 12), bg="#f2f2f2", fg="blue").pack(pady=10)
    image_label = tk.Label(root, bg="#f2f2f2")
    image_label.pack(pady=10)
    explanation_box = scrolledtext.ScrolledText(root, width=80, height=15)
    explanation_box.pack(pady=10)
    download_button = tk.Button(root, text="Download Report (PDF)", bg="#2196F3", fg="white", font=("Arial", 12), state=tk.DISABLED)
    download_button.pack(pady=10)

    root.mainloop()

# ================== Run GUI =======================
if __name__ == "__main__":
    run_gui()


1/1 [==============================] - 0s 44ms/step
